# Week 1: Image Recognition

In this notebook we'll be exploring a the image classification task using convolution neural networks.  We'll be building solutions using both pre-trained models and building our own. 

In [1]:
%matplotlib inline
import tensorflow as tf
from tensorflow import ConfigProto as config

# Import our class, and instantiate
from keras.preprocessing import image
from keras.optimizers import rmsprop
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

from keras.applications.vgg19 import VGG19

Using TensorFlow backend.


In [2]:
# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=64

# path = "D://Kaggle Data//dogs cats//"
path = "D://Kaggle Data//dogs cats//sample//"

In [3]:
"""Using a class called ImageDataGenerator both as a utility for loading in raw images, but also to do 
some of the necessary data augmentation to boost generalizability.  The options are documented here:
https://keras.io/preprocessing/image/
"""

train_datagen = image.ImageDataGenerator()
validation_datagen = image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
                path+'train',
                target_size=(150, 150),
                batch_size=batch_size,
                class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
                path+'valid',
                target_size=(150, 150),
                batch_size=batch_size,
                class_mode='binary')

Found 200 images belonging to 2 classes.
Found 48 images belonging to 2 classes.


In [ ]:
base_model = VGG19(weights='imagenet', include_top=False, classes=2)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
prediction = Dense(1, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=prediction)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='binary_crossentropy')   

# train the model on the new data for a few epochs
model.fit_generator(
        train_generator,
        steps_per_epoch = 2000,
        epochs = 10,
        validation_data=validation_generator,
        validation_steps=800)

Epoch 1/10


In [9]:
train_generator.image_shape

(150, 150, 3)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/50
